In [15]:
import pandas as pd
import matplotlib as plt
import numpy as np

fake = pd.read_csv("fake.csv",)
true = pd.read_csv("true.csv",)

fake['fake'] = 1
true['fake'] = 0

df = [fake, true]

data = pd.concat(df)

data.head()


,title,text,subject,date,fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [16]:
fake.isnull().sum()
true.isnull().sum()

title      0
text       0
subject    0
date       0
fake       0
dtype: int64

In [17]:
data.drop_duplicates(inplace=True)
duplicates = data[data.duplicated()]
duplicates

,title,text,subject,date,fake


In [18]:
data.drop(['text', 'subject', 'date'], axis=1, inplace=True)

data.head()

,title,fake
0,Donald Trump Sends Out Embarrassing New Year’...,1
1,Drunk Bragging Trump Staffer Started Russian ...,1
2,Sheriff David Clarke Becomes An Internet Joke...,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,1
4,Pope Francis Just Called Out Donald Trump Dur...,1


In [19]:
data['title'] = data['title'].apply(lambda x: x.lower())
data.head()

,title,fake
0,donald trump sends out embarrassing new year’...,1
1,drunk bragging trump staffer started russian ...,1
2,sheriff david clarke becomes an internet joke...,1
3,trump is so obsessed he even has obama’s name...,1
4,pope francis just called out donald trump dur...,1


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)

from sklearn.model_selection import train_test_split

x = data['title']
y = data['fake']

x_train, x_test, y_train, y_test =train_test_split(x,y, test_size=0.3, random_state=42)

x_train = tfidf.fit_transform(x_train)
x_test = tfidf.transform(x_test)

In [21]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', C=1)
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)




In [22]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score * 100, 2)}%')

Accuracy: 94.79%


In [23]:
from sklearn.metrics import  confusion_matrix
print(confusion_matrix(y_test, y_pred)) 

[[5993  290]
 [ 408 6716]]


In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      6283
           1       0.96      0.94      0.95      7124

    accuracy                           0.95     13407
   macro avg       0.95      0.95      0.95     13407
weighted avg       0.95      0.95      0.95     13407



In [25]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import PassiveAggressiveClassifier

pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(x_train, y_train)
y_pred = pac.predict(x_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score * 100, 2)}%')
print(confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test, y_pred))

Accuracy: 93.95%
[[5871  412]
 [ 399 6725]]
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      6283
           1       0.94      0.94      0.94      7124

    accuracy                           0.94     13407
   macro avg       0.94      0.94      0.94     13407
weighted avg       0.94      0.94      0.94     13407



In [26]:
test_data = "Trump sides with saudis over afganistan"


models = [pac, svm]

for model in models:
    corpus = [] # corpus is a colection of records
    corpus.append(test_data)
    corpus = tfidf.transform(corpus)
    pred = model.predict(corpus)
    print(pred)
    if pred == 1:
        print(test_data, "is fake news!")
    else:
        print(test_data, "is real news")

[0]
Trump sides with saudis over afganistan is real news
[0]
Trump sides with saudis over afganistan is real news


In [27]:
import pickle

pickle.dump(svm, open('model.pkl', 'wb'))
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


df['Positive'].value_counts().plot(kind='bar', color='cornflowerblue')


TypeError: list indices must be integers or slices, not str